<a href="https://colab.research.google.com/github/Kushvanth015/fmml_labs/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


#**1st question answer**

Increase the Percentage of Validation Set:

Higher Validation Set Accuracy: When you increase the percentage of the validation set, you allocate more data for validation. This often leads to a more reliable and representative estimate of your model's performance on unseen data. In other words, your validation accuracy tends to be a better reflection of how your model will perform on new, unseen data.

Smaller Training Set: However, the trade-off is that you have a smaller training set. With less training data, your model may have a harder time learning the underlying patterns in the data. This can result in a potential decrease in training set accuracy and may make it more challenging to train complex models.

#**2nd question answer**

Larger Training Set:

When you have a larger training set, your model has more data to learn from. This typically results in a model that can capture more of the underlying patterns in the data, leading to better training set accuracy.
A model trained on a larger training set may generalize better to the validation set because it has learned more about the data distribution. As a result, validation set accuracy tends to be a reasonable estimate of how well the model will perform on unseen data, such as the test set.
Smaller Training Set:

With a smaller training set, your model may struggle to capture the complexity of the data. It's more prone to overfitting, where it memorizes the training data instead of learning meaningful patterns. This can lead to high training set accuracy but poor generalization to the validation and test sets.
Validation set accuracy may not reliably predict test set accuracy when the training set is small. The model's performance on the validation set can be misleading, as it may overfit to the validation data, resulting in an overly optimistic estimate of generalization performance.

#**3rd question answer**

Small Datasets:

In cases where you have a small dataset (e.g., a few hundred or fewer examples), it's common to allocate a relatively larger percentage for the validation set. A split like 20-30% for validation can help ensure that you have a reasonable amount of data for model evaluation.
Larger Datasets:

For larger datasets, you can allocate a smaller percentage to the validation set. A common choice is 10-20% or even less, as there is more data available for training, and you can afford to set aside a smaller portion for validation.
Complex Models:

If you are working with complex models (e.g., deep neural networks with many parameters), you may want to allocate a larger validation set to help ensure that the model generalizes well. Complex models have a higher capacity to overfit the training data, so a larger validation set can be beneficial.

#**1st question answer**

Reduced Variance: Cross-validation helps reduce the variance in your performance estimate. When you split your dataset into multiple subsets (folds) and train/validate your model on different combinations of these subsets, you obtain multiple performance metrics. Averaging these metrics provides a more stable estimate of model performance by reducing the influence of random data variations.

Better Representativeness: Different random splits of your data can lead to varying validation results. By averaging these results, you ensure that your performance estimate is not overly influenced by a single, possibly unrepresentative, validation split.

#**2nd question answer**

Cross-validation is Still on Training Data: In cross-validation, you are partitioning your dataset into training and validation subsets, but these subsets are still drawn from your original dataset. They are not entirely independent of the training data. Therefore, while cross-validation provides a more robust estimate of how well your model generalizes to unseen data, it's not the same as testing on entirely new, unseen data.

Assumption of Stationarity: Cross-validation assumes that the data distribution is stationary, meaning that the underlying data distribution does not change significantly between different folds. If this assumption is violated, cross-validation results may not accurately reflect how the model will perform on new data.

#**3rd question answer**

Higher Iterations (More Folds):

Pros: Using more folds (higher iterations) can provide a more accurate estimate of model performance. With more iterations, you're essentially using a larger portion of your data for validation, reducing the variability in the estimate.
Cons: Using more folds can be computationally expensive, especially if your dataset is large or your model is complex. It may also lead to longer training times, as the model needs to be trained and validated multiple times.
Lower Iterations (Fewer Folds):

Pros: Fewer folds are computationally faster, which can be advantageous when you have limited computational resources or need to perform cross-validation quickly.
Cons: With fewer iterations, you have a smaller sample size for estimating model performance. This can result in higher variability in the estimates, making them less reliable. It may also lead to a less accurate representation of how well your model generalizes to unseen data.

#**4th question answer**

Small Training Dataset:

If your training dataset is very small, increasing the number of iterations in cross-validation can help by allowing your model to be trained on more diverse subsets of the data. This can potentially lead to a more stable and reliable estimate of model performance.
Small Validation Dataset:

If your validation dataset is small, increasing the number of iterations in cross-validation can also be beneficial. With more iterations, you're using different portions of the data for validation, which can reduce the impact of having a small validation set.
However, there are limitations to consider:

Diminishing Returns: While increasing the number of iterations can improve the robustness of your estimate, there are diminishing returns. Eventually, the gains in estimate reliability may not justify the increased computational cost.

Model Complexity: If your model is very complex, it may require a relatively larger training dataset to generalize effectively. Cross-validation can help, but it can't fundamentally address the limitations of having an extremely small training dataset.

Bias-Variance Trade-off: Increasing the number of iterations can reduce the variance in your estimate, but it may introduce bias. If your dataset is extremely small, repeatedly splitting it into smaller subsets for validation can lead to overly optimistic or pessimistic estimates.